In [ ]:
# Подключаемся к Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/

/content


In [ ]:
# Устанавливаем библиотеки
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
#
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

#Разбиваем данные
file_path = '/content/drive/MyDrive/Laptop_learn/Laptop_price.csv'
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()
num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])


pipeline = Pipeline([
    ('preprocessor', preprocessor),
   ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, '/content/laptop_price_model.pkl')

['/content/laptop_price_model.pkl']

In [ ]:
# Инициализируем локальный репозиторий в Google Colab:
!git init

# Добавляем файлы в репозиторий:
!git add /content/drive/MyDrive/Laptop_learn
!git add '/content/drive/MyDrive/Colab Notebooks/laptop_ml.ipynb'

!git config --global user.email "madvocato@protonmail.com"
!git config --global user.name "madvocato"

# Получаем токен из среды для работы с Github
from google.colab import userdata
github_token = userdata.get('GITHUB_TOKEN')

# Первый коммит:
!git commit -m 'Добавлен ML-пайплайн'

!git branch -M main

repo_url = f"https://{github_token}@github.com/Madvocato/laptop_price_ml.git"

# Cвязываем репозиторий с GitHub и отправляем файлы:
!git remote add origin {repo_url}
!git push -u origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
[master (root-commit) 34cf042] Добавлен ML-пайплайн
 3 files changed, 1002 insertions(+)
 create mode 100644 drive/MyDrive/Colab Notebooks/laptop_ml.ipynb
 create mode 100644 drive/MyDrive/Laptop_learn/Laptop_price.csv
 create mode 100644 drive/MyDrive/Laptop_learn/laptop_price_model.pkl
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (9/9), 9

In [ ]:
#Создаем FastAPI-приложение

%%writefile app.py
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO

app = FastAPI()

# Загрузка обученной модели
model_path = "laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}

Overwriting app.py


In [ ]:
!pip install python-multipart

In [ ]:
from google.colab import userdata
ngrok_token = userdata.get('NGROK_TOKEN')

!ngrok config add-authtoken {ngrok_token}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &

In [ ]:
!uvicorn app:app --host 0.0.0.0 --port 8000 --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [34300] using StatReload
INFO:     Started server process [34302]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [34302]
INFO:     Stopping reloader process [34300]


In [ ]:
from pyngrok import ngrok # Подключаем публичный URL
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)

API доступно по адресу: NgrokTunnel: "https://8430-34-106-108-207.ngrok-free.app" -> "http://localhost:8000"


In [171]:
from pyngrok import ngrok

tunnels = ngrok.get_tunnels()
print("Активные туннели:", tunnels)

Активные туннели: [<NgrokTunnel: "https://8430-34-106-108-207.ngrok-free.app" -> "http://localhost:8000">]


In [ ]:
!cat fastapi.log


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [34640] using StatReload
INFO:     Started server process [34642]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [ ]:
!git add /content/drive/MyDrive/Laptop_learn
!git add '/content/drive/MyDrive/Colab Notebooks/laptop_ml.ipynb'
!git add laptop_price_model.pkl
!git add app.py
!git commit -m "Добавлено FastAPI-приложение"
!git push origin main

[main b34e05f] Добавлено FastAPI-приложение
 4 files changed, 35 insertions(+), 1 deletion(-)
 create mode 100644 app.py
 rewrite drive/MyDrive/Colab Notebooks/laptop_ml.ipynb (97%)
 create mode 100644 drive/MyDrive/Laptop_learn/app.py
 create mode 100644 laptop_price_model.pkl
Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (9/9), 7.32 KiB | 3.66 MiB/s, done.
Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), done.
To https://github.com/Madvocato/laptop_price_ml.git
   34cf042..b34e05f  main -> main
